In [ ]:
import pandas
import os
import numpy
import plotly.express
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef, average_precision_score
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import xgboost
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
import glob

# Créez un DataFrame vide
df = []

# Chemin vers le répertoire contenant vos fichiers CSV
csv_files = glob.glob('data/Physical dataset/*.csv')
print(csv_files)
# Parcourez la liste de fichiers CSV et ajoutez-les au DataFrame
for csv_file in csv_files:
    delimiter = '\t'
    encoding="utf-16le"
    if csv_file.endswith("4.csv"):
        delimiter = ','
        encoding = "utf-8"
    df_temp = pandas.read_csv(csv_file, encoding=encoding, delimiter=delimiter)
    df.append(df_temp)

df  = pandas.concat(df, ignore_index=True)
# Maintenant, df contient l'ensemble des données de tous les fichiers CSV


In [ ]:
import pandas as pd
import numpy as np

# Remplacer les valeurs non nulles dans Label_n par celles de Lable_n
df['Label_n'] = df['Label_n'].combine_first(df['Lable_n'])

# Supprimer la colonne Lable_n
df.drop('Lable_n', axis=1, inplace=True)


In [ ]:
nb_attaque= df['Label'].value_counts()
print(nb_attaque)
ratio = (nb_attaque.iloc[1] + nb_attaque.iloc[2]+ nb_attaque.iloc[3] + nb_attaque.iloc[5])/(nb_attaque.iloc[0]+nb_attaque.iloc[1]+nb_attaque.iloc[2]+ nb_attaque.iloc[3] + nb_attaque.iloc[4] +nb_attaque.iloc[5])
print(ratio*100)

Ici on test sur des partie du dataframe pour voir le nombre minimal de data à prendre pour avoir un bon F1 score

In [ ]:
# Create reduced datasets
train_sizes = [100, 500,1000,5000,10000]
reduced_datasets = {}
# Define the sizes of X_train you want to experiment with

# Initialize lists to store results

recall_scores = []
accuracy_scores = []

clf = RandomForestClassifier()

#on drop Label, Time et Label_n pour pouvoir faire l'apprentissage
#Label_n met aussi les erreur physique comme attaque, or ce ne sont pas des attaques. C'est pourquoi on utilise pas cette colonne et on créer nous même une colonne.


for size in train_sizes:

    fraudulent_entries = df[df['Label_n'] == 1].sample(n=int(size * ratio) +1)
    non_fraudulent_entries = df[df['Label_n'] == 0].sample(n=int(size * (1-ratio)))

    # Concatenate both sets to create df_reduced
    df_reduced = pandas.concat([fraudulent_entries, non_fraudulent_entries])

    # Store the reduced dataset
    reduced_datasets[size] = df_reduced
    Y_red=df_reduced['Label_n'].copy()
    X_red=df_reduced.drop(['Label_n', 'Time','Label'], axis=1)
    # Optionally, split the reduced dataset into training and test sets
    X_train, X_test, Y_train, Y_test = train_test_split(X_red, Y_red, test_size=0.2, random_state=42)

    # Create an RF classifier
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    Y_pred_proba = clf.predict_proba(X_test)
    # Evaluate the RF classifier
   
    recall_scores.append(recall_score(Y_test, Y_pred))
    accuracy_scores.append(accuracy_score(Y_test, Y_pred))

# Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(train_sizes, recall_scores, label='Recall')
plt.plot(train_sizes, accuracy_scores, label='Accuracy')

plt.title('Random forest Classifier Performance for Different Training Set Sizes')
plt.xlabel('Training Set Size')
plt.ylabel('Score')
plt.legend()
plt.show()
   

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import plotly.express as px

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier()

# Train the classifier on the training data
rf_classifier.fit(X_train, Y_train)

# Make predictions on the test data
Y_pred_rf = rf_classifier.predict(X_test)
# Evaluate the Random Forest classifier
print('Random Forest Classifier Metrics:')
print('Confusion matrix:', confusion_matrix(Y_test, Y_pred_rf))
print('Precision:', precision_score(Y_test, Y_pred_rf))
print('Recall:', recall_score(Y_test, Y_pred_rf))
print('Accuracy:', accuracy_score(Y_test, Y_pred_rf))
print('Balanced accuracy:', balanced_accuracy_score(Y_test, Y_pred_rf))
print('Matthews Correlation Coefficient:', matthews_corrcoef(Y_test, Y_pred_rf))
print('AUPRC:', average_precision_score(Y_test, Y_pred_rf))

# Visualize the confusion matrix
fig_rf = px.imshow(confusion_matrix(Y_test, Y_pred_rf), title='Random Forest Confusion matrix')
fig_rf.show()